In [1]:
!pip install topmost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.6/93.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 22.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.6/89.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 9.9 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 9.6 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 63.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 48.1 MB/s eta 0:00:0000:01:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.5/309.5 kB 18.9 

In [2]:
import topmost
import torch
from torch import nn
import torch.nn.functional as F
from topmost.models.dynamic.DETM import DETM
from topmost import eva
import numpy as np
import itertools
import json
import joblib
from tqdm import tqdm
import os
import json
import random

ModuleNotFoundError: No module named 'numpy.rec'

In [ ]:
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
from topmost.data.file_utils import split_text_word

## Funciones externas

In [ ]:
def get_theta_redefined(self, bows, times, eta=None): ## amortized inference
        """Returns the topic proportions.
        """
        # normalized_bows = bows / bows.sum(1, keepdims=True)
        # print("🍀 Before", normalized_bows)

        normalized_bows = bows / (bows.sum(1, keepdims=True) + 1e-12)
        # print("🍀 After", normalized_bows)
    
        if eta is None and self.training is False:
            eta, kl_eta = self.get_eta(self.rnn_inp)

        eta_td = eta[times]
        inp = torch.cat([normalized_bows, eta_td], dim=1)
        q_theta = self.q_theta(inp)
        if self.enc_drop > 0:
            q_theta = self.t_drop(q_theta)
        mu_theta = self.mu_q_theta(q_theta)
        logsigma_theta = self.logsigma_q_theta(q_theta)
        z = self.reparameterize(mu_theta, logsigma_theta)
        theta = F.softmax(z, dim=-1)
        kl_theta = self.get_kl(mu_theta, logsigma_theta, eta_td, torch.zeros(self.num_topics).to(self.device))

        if self.training:
            return theta, kl_theta
        else:
            return theta

DETM.get_theta = get_theta_redefined

In [ ]:
class DETMFixed(DETM):
    def __init__(self, vocab_size, num_times, train_size, train_time_wordfreq, num_topics=50, train_WE=True, pretrained_WE=None, en_units=800, eta_hidden_size=200, rho_size=300, enc_drop=0.0, eta_nlayers=3, eta_dropout=0.0, delta=0.005, theta_act='relu', device='cpu'):
        super().__init__(vocab_size, num_times, train_size, train_time_wordfreq, num_topics, True, pretrained_WE, en_units, eta_hidden_size, rho_size, enc_drop, eta_nlayers, eta_dropout, delta, theta_act, device)
        
        ## define the word embedding matrix \rho
        if not self.train_WE:
            rho = nn.Embedding(pretrained_WE.shape[1], pretrained_WE.shape[0])
            rho.weight.data = torch.from_numpy(pretrained_WE)
            self.rho = rho.weight.data.clone().float().to(self.device)

In [ ]:
def _coherence_modified(reference_corpus, vocab, top_words, cv_type='c_v'):
    # print('🍀')
    
    split_top_words = split_text_word(top_words)
    num_top_words = len(split_top_words[0])
    for item in split_top_words:
        assert num_top_words == len(item)

    split_reference_corpus = split_text_word(reference_corpus)
    dictionary = Dictionary(split_text_word(vocab))

    cm = CoherenceModel(texts=split_reference_corpus, dictionary=dictionary, topics=split_top_words, topn=num_top_words, coherence=cv_type)
    cv_per_topic = cm.get_coherence_per_topic()
    # print(f"Coherence scores per topic: {cv_per_topic}")

    valid_scores = [score for score in cv_per_topic if not np.isnan(score)]
    if not valid_scores:
        raise ValueError("All coherence scores are NaN.")
    score = np.mean(valid_scores)

    return score

In [ ]:
eva.topic_coherence._coherence = _coherence_modified

def eval(dataset, trainer, top_words):
    # get theta (doc-topic distributions)
    train_theta, test_theta = trainer.export_theta()

    train_times = dataset.train_times.cpu().numpy()
    # compute topic coherence
    dynamic_TC = eva.dynamic_coherence(dataset.train_texts, train_times, dataset.vocab, top_words, verbose=True)
    print("dynamic_TC: ", dynamic_TC)

    # compute topic diversity
    dynamic_TD = eva.dynamic_diversity(top_words, dataset.train_bow.cpu().numpy(), train_times, dataset.vocab, verbose=False)
    print("dynamic_TD: ", dynamic_TD)

    # evaluate clustering
    clustering = eva._clustering(test_theta, dataset.test_labels)
    print(clustering)

    # evaluate classification
    classification = eva._cls(train_theta, test_theta, dataset.train_labels, dataset.test_labels)
    print(classification)
    
    json = {
        "dynamic_TC": dynamic_TC,
        "dynamic_TD": dynamic_TD,
        "clustering": clustering,
        "classification": classification
    }
    
    return json, dynamic_TC, dynamic_TD, clustering, classification

In [ ]:
import joblib
import json

def save_txt(obj, path):
    with open(path, 'w') as file:
        file.write(obj)

def save_top_words_txt(top_words, path):
    with open(os.path.join(path, 'top_words.txt'), 'w') as file:
        for i, time in enumerate(top_words):
            file.write(f'================= Time {i} ================= \n')
            for j, topic in enumerate(time):
                file.write(f'================= Topic {j} ================= \n')
                file.write(f'{topic}')
                file.write('\n')
            file.write('\n')

def save_json(obj, path):
    with open(path, 'w') as file:
        json.dump(obj, file)

def save_joblib(obj, path):
    joblib.dump(obj, path)

def get_desc(num_topics, rho_size, en_units, eta_hidden_size, enc_drop, eta_nlayers, eta_dropout, delta, theta_act, learning_rate, epochs):
    return f'Trainig model with hyperparameters: \n num_topics (K):  {num_topics} rho_size: {rho_size} \n en_units: {en_units} \n eta_hidden_size: {eta_hidden_size} \n enc_drop: {eta_dropout} \n eta_nlayers: {eta_nlayers} \n eta_dropou: {eta_dropout} \n delta: {delta} \n theta_act: {theta_act} \n learning_rate: {learning_rate} \n epochs: {epochs}'

def get_hyperparameters_json(config):
    return { 
        "num_topics" : config['num_topics'],
        "rho_size": config['rho_size'],
        "en_units" : config['en_units'],
        "eta_hidden_size" : config['eta_hidden_size'],
        "enc_drop": config['enc_drop'],
        "eta_nlayers" : config['eta_nlayers'],
        "eta_dropou" : config['eta_dropout'],
        "delta" : config['delta'],
        "theta_act" : config['theta_act'],
        "learning_rate": config['learning_rate'],
        "epochs": config['epochs']}
    
# def get_hyperparameter_space():
    # with open('./hyperparameter_space.json', 'r') as f:
        # return json.load(f)

def get_hyperparameter_space():
    #with open("hyperparameter_space.json", 'r') as f:
        #return json.load(f)
    return {
        "num_topics": [20, 50, 100],
        "rho_size": [200, 300, 400],
        "en_units": [400, 800, 1200],
        "eta_nlayers": [2, 3, 4],
        "delta": [0.001, 0.005, 0.01, 0.05],
        "theta_act": ['anh', 'softplus', 'relu', 'rrelu', 'leakyrelu', 'elu', 'selu', 'glu'],
        "eta_hidden_size": [100, 200, 300],
    
        # Parametros del optimizador
        
        "enc_drop": [0.0, 0.2, 0.3, 0.5],
        "eta_dropout": [0.0, 0.2, 0.3, 0.5],
        "learning_rate": [1e-3, 5e-4, 0.02],
        "epochs": [200, 400, 800]
    }

def get_default_config():
    return { 
        "num_topics" : 50,
        "rho_size": 300,
        "en_units" : 800,
        "eta_hidden_size" : 200,
        "enc_drop": 0.0,
        "eta_nlayers" : 3,
        "eta_dropout" : 0.0,
        "delta" : 0.005,
        "theta_act" : 'relu',
        "learning_rate": 0.005,
        "epochs": 100}

def random_configuration(hyperparameter_space):
    return {
        'num_topics': random.choice(hyperparameter_space["num_topics"]),
        'rho_size': random.choice(hyperparameter_space["rho_size"]),
        'en_units': random.choice(hyperparameter_space["en_units"]),
        'eta_hidden_size': random.choice(hyperparameter_space["eta_hidden_size"]),
        'enc_drop': random.choice(hyperparameter_space["enc_drop"]),
        'eta_nlayers': random.choice(hyperparameter_space["eta_nlayers"]),
        'eta_dropout': random.choice(hyperparameter_space["eta_dropout"]),
        'delta': random.choice(hyperparameter_space["delta"]),
        'theta_act': random.choice(hyperparameter_space["theta_act"]),
        'learning_rate': random.choice(hyperparameter_space["learning_rate"]),
        'epochs': random.choice(hyperparameter_space["epochs"])}

### --------------------------------------------------------

In [ ]:
def save_result(top_words, trainer, config, metrics_json):
    global samples 
    path = os.path.join(output_dir, f'{samples}')
    os.makedirs(path, exist_ok=True)

    try:
        save_top_words_txt(top_words, path)
        save_json(config, os.path.join(path, 'hyperparameters.json'))
        save_json(metrics_json, os.path.join(path, 'metrics_json.json'))
        save_joblib(trainer, os.path.join(path, 'trainer.joblib'))
        save_joblib(top_words, os.path.join(path, 'top_words.joblib'))
    except Exception as e:
        traceback.print_exc()

In [ ]:
import traceback

def evaluate_configuration(config):
    global samples, tested_configs, output_dir

    if config.values() in tested_configs.keys():
        return tested_configs[tuple(config.values())]
    
    model = DETMFixed(
            num_topics=config['num_topics'], 
            vocab_size=dataset.vocab_size,
            num_times=dataset.num_times,
            train_size=dataset.train_size,
            train_time_wordfreq=dataset.train_time_wordfreq,
            train_WE=False, 
            pretrained_WE=dataset.pretrained_WE, 
            en_units=config['en_units'], 
            eta_hidden_size=config['eta_hidden_size'], 
            rho_size=config['rho_size'], 
            enc_drop=config['enc_drop'], 
            eta_nlayers=config['eta_nlayers'], 
            eta_dropout=config['eta_dropout'], 
            delta=config['delta'], 
            theta_act=config['theta_act'],
            device=device)

    model = model.to(device)
    # trainer = topmost.DynamicTrainer(model, dataset, epochs=config['epochs'], learning_rate=config['learning_rate'], verbose=False)

    try:
        trainer = topmost.DynamicTrainer(model, dataset, epochs=config['epochs'], learning_rate=config['learning_rate'], verbose=False)
        top_words, train_theta = trainer.train()
        metrics_json, dynamic_TC, dynamic_TD, clustering, classification = eval(dataset, trainer, top_words)

        save_result(top_words, trainer, config, metrics_json)

        samples += 1
    
        # return 0.4 * dynamic_TC + 0.4 * dynamic_TD + 0.1 * clustering + 0.1 * classification
        result = 0.5 * dynamic_TC + 0.5 * dynamic_TD

    except Exception as e:
        # print(f'❌ Error config {config}')
        traceback.print_exc()
        result = float('-inf')

    tested_configs[tuple(config.values())] = result
    joblib.dump(tested_configs, os.path.join(output_dir, 'tested_configs.joblib'))
    return result
 

In [ ]:
def evolutionary_search(search_space, generations=10, mutation_rate=0.1, population_size=20, population=None, generation_count=0):
    global samples
    
    if not population:
        population = [random_configuration(search_space) for _ in range(population_size - 1)]
        population.append(get_default_config())

    # print(population)
    
    for generation in range(generation_count, generations):
        scores = [(config, evaluate_configuration(config)) for config in population]
        scores.sort(key=lambda value: value[1], reverse=True)  

        joblib.dump(scores[0], f'best_per_generation_path_{generation}.joblib')
        print(f"🧬 Generación {generation + 1}, mejor resultado: {scores[0][1]:.4f}")
        
        num_parents = population_size // 2
        parents = [config for config, _ in scores[:num_parents]]
        
        children = []
        while len(children) < population_size - num_parents:
            parent_1, parent_2 = random.sample(parents, 2)
            child = {key: random.choice([parent_1[key], parent_2[key]]) for key in search_space.keys()}
            
            if random.random() < mutation_rate:
                param_to_mutate = random.choice(list(search_space[mutate_model_training].keys()))
                print(param_to_mutate)
                child[param_to_mutate] = random.choice(search_space[param_to_mutate])
            
            children.append(child)
        
        population = parents + children
        joblib.dump(population, os.path.join(output_dir, 'population.joblib'))
    
    best_config, best_score = max(scores, key=lambda x: x[1])
    return best_config, best_score

## Main

In [ ]:
device = "cuda"  # or "cpu"
dataset_dir = "/kaggle/input/revista-ciencias-mdicas-de-la-habana-cmed"
output_dir = "./results"
os.makedirs(output_dir, exist_ok=True)
dataset = topmost.data.DynamicDataset(dataset_dir, batch_size=200, read_labels=True, device=device)

In [ ]:
output_dir = "./results"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
def load_json(path):
    with open(os.path.join(path, 'metrics_config.json'), "r") as file:
        return json.load(file)

In [ ]:
import os
import json

results_path = '/kaggle/working/results'
results_path = [dir_.path for dir_ in os.scandir(results_path) if not dir_.is_file()]

print(len(results_path))

In [ ]:
# import joblib

# population_size = 50
# # population = None
# population = [load_json(path) for path in results_path]
# population.sort(key=lambda value: value["metrics"]["dynamic_TC"] + value["metrics"]["dynamic_TD"], reverse=True)
# population = [item["config"] for item in population[:population_size]]
# print(population)
# joblib.dump(population, os.path.join(output_dir, 'population.joblib'))

In [ ]:
import joblib
population_size = 50
# population = None
population = joblib.load(os.path.join(output_dir, 'population.joblib'))

In [ ]:
import joblib
# tested_configs = dict()
tested_configs = joblib.load('/kaggle/working/results/tested_configs.joblib')
# print(tested_configs)

In [ ]:
best_per_generation_path = './best_per_generation_path'
os.makedirs(best_per_generation_path, exist_ok=True)

In [ ]:
generation = len([dir_.path for dir_ in os.scandir(best_per_generation_path) if dir_.is_file()])

In [ ]:
samples = len(results_path)

In [ ]:
hyperparameter_space = get_hyperparameter_space()

best_config, best_score = evolutionary_search(hyperparameter_space, population_size=population_size, generations=100, mutation_rate=0.2, population=population, generation_count=generation)
print(best_config)
print(best_score)

## Evaluando resultados

In [ ]:
import os

results_path = '/kaggle/working/results'
results_path = [dir_.path for dir_ in os.scandir(results_path) if not dir_.is_file()][0]
# results_path = [dir_.path for dir_ in os.scandir(results_path) if not dir_.is_file()]

# print(results_path)

In [ ]:
import json
for result in results_path:
    with open(os.path.join(result, 'hyperparameters.json'), 'r') as file:
        config = json.load(file)

    with open(os.path.join(result, 'metrics_json.json'), 'r') as file:
        metrics = json.load(file)

    with open(os.path.join(result, 'metrics_config.json'), 'w') as file:
        json.dump({"config": config, "metrics": metrics}, file)

In [ ]:
file = 'trainer.joblib'
    
for path in results_path:
    file_path = os.path.join(path, file)
    if os.path.isfile(file_path):
        os.remove(file_path)

In [ ]:
!zip -r results.zip /kaggle/working/results

In [ ]:
old_tc = float('-inf')
TC = []
old_td = float('-inf')
TD = []
old_tc_td = float('-inf')
TC_TD = []

for path in results_path:
    with open(os.path.join(path, 'metrics_json.json'), 'r') as file:
        metrics = json.load(file)

    cr_tc = metrics["dynamic_TC"]
    if cr_tc >= old_tc:
        with open(os.path.join(path, 'metrics_config.json'), 'r') as file:
            new = json.load(file)
        if cr_tc == old_tc:
            TC.append((new, path))
        else:
            old_tc = cr_tc
            TC = [(new, path)]

    cr_td = metrics["dynamic_TD"]
    if cr_td >= old_td:
        with open(os.path.join(path, 'metrics_config.json'), 'r') as file:
            new = json.load(file)
        if cr_td == old_td:
            TD.append((new, path))
        else:
            old_td = cr_td
            TD = [(new, path)]

    cr_tc_td = 0.5 * cr_tc + 0.5 * cr_td
    if cr_tc_td >= old_tc_td:
        with open(os.path.join(path, 'metrics_config.json'), 'r') as file:
            new = json.load(file)
        if cr_tc_td == old_tc_td:
            TC_TD.append((new, path))
        else:
            old_tc_td = cr_tc_td
            TC_TD = [(new, path)]

print(TC)
print(len(TC))
print('-------------------------------------')
print(TD)
print(len(TD))
print('-------------------------------------')
print(TC_TD)
print(len(TC_TD))

best_results_path = './best_results'
os.makedirs(best_results_path, exist_ok=True)
save_json(TC, os.path.join(best_results_path, 'best_tc.json'))
save_json(TD, os.path.join(best_results_path, 'best_td.json'))
save_json(TC_TD, os.path.join(best_results_path, 'best_tc_td.json'))

In [ ]:
!zip -r best_results/best_results.zip /kaggle/working/results/45

## Obteniendo theta y beta del modelo con la mejor configuracion

In [ ]:
import topmost

In [ ]:
device = "cpu" # or"cuda"
dataset_dir = "/kaggle/input/revista-ciencias-mdicas-de-la-habana-cmed"
dataset = topmost.data.DynamicDataset(dataset_dir, batch_size=200, read_labels=True, device=device)

In [ ]:
best_results_path = './best_results'

In [ ]:
    config = {'num_topics': 20, 'rho_size': 400, 'en_units': 1200, 'eta_hidden_size': 300, 'enc_drop': 0.2, 'eta_nlayers': 4, 'eta_dropout': 0.0, 'delta': 0.001, 'theta_act': 'selu', 'learning_rate': 0.0005, 'epochs': 400}
    model = DETMFixed(
            num_topics=config['num_topics'], 
            vocab_size=dataset.vocab_size,
            num_times=dataset.num_times,
            train_size=dataset.train_size,
            train_time_wordfreq=dataset.train_time_wordfreq,
            train_WE=False, 
            pretrained_WE=dataset.pretrained_WE, 
            en_units=config['en_units'], 
            eta_hidden_size=config['eta_hidden_size'], 
            rho_size=config['rho_size'], 
            enc_drop=config['enc_drop'], 
            eta_nlayers=config['eta_nlayers'], 
            eta_dropout=config['eta_dropout'], 
            delta=config['delta'], 
            theta_act=config['theta_act'],
            device=device)

    model = model.to(device)
    # trainer = topmost.DynamicTrainer(model, dataset, epochs=config['epochs'], learning_rate=config['learning_rate'], verbose=False)

    try:
        trainer = topmost.DynamicTrainer(model, dataset, epochs=config['epochs'], learning_rate=config['learning_rate'], verbose=False)
        top_words, train_theta = trainer.train()
        metrics_json, dynamic_TC, dynamic_TD, clustering, classification = eval(dataset, trainer, top_words)
        save_joblib(trainer, best_results_path)
    except Exception as e:
        # print(f'❌ Error config {config}')
        traceback.print_exc()
        result = float('-inf')

In [ ]:
save_joblib(trainer, os.path.join(best_results_path, 'trainer.joblib'))

In [ ]:
save_joblib(model, os.path.join(best_results_path, 'model.joblib'))

In [ ]:
top_words = joblib.load('/kaggle/working/results/45/top_words.joblib')
print(type(top_words))
print(len(top_words))
print(top_words)

In [ ]:
train_theta, test_theta = trainer.export_theta()
print(type(train_theta))
print(train_theta)

In [ ]:
beta = model.get_beta()
# print(topic_matrix)
print(beta.shape[0]) # Numero de time_slices
print(beta.shape[1]) # Numero de tópicos
print(beta.shape[2]) # Numero de palabras

 ## Segun beta (matriz de topicos) ver la probabilidad de cada palabra en cada topico en cada time slice

In [ ]:
import os
vocab_path = '/kaggle/input/revista-ciencias-mdicas-de-la-habana-cmed/vocab.txt'
with open(vocab_path, 'r') as file:
    words = file.readlines()
    vocab = [word.replace('\n', '') for word in words]

In [ ]:
times_path = '/kaggle/input/revista-ciencias-mdicas-de-la-habana-cmed/time2id.txt'
with open(times_path, 'r') as file:
    times = file.readlines()[0]
print(times)

In [ ]:
timelist = list(json.loads(times.replace('\'', '\"')).keys())
timelist = [int(time) for time in timelist]
print(type(timelist))
print(timelist)

In [ ]:
import numpy

print(numpy.__version__)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt 

def get_word_index(words):
    return [vocab.index(word) for word in words]

def get_words(words_index):
    return [vocab[index] for index in words_index]

def get_betas(word_index, topic):
    return [beta[time][topic][word_index].item() for time in range(len(timelist))]

def graphic(words, betas, topic_name):
    T = len(timelist)
    fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(18, 9), dpi=80, facecolor='w', edgecolor='k')
    ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8 = axes.flatten()
    ticks = [str(x) for x in timelist]
    for i, comp in enumerate(betas):
        ax1.plot(range(len(timelist)), comp, label=words[i], lw=2, linestyle='--', marker='o', markersize=4)
    ax1.legend(frameon=False)
    print('np.arange(T)[0::10]: ', np.arange(T)[0::10])
    ax1.set_xticks(np.arange(len(timelist)))
    ax1.set_xticklabels(timelist)
    ax1.set_title(topic_name, fontsize=12)

    plt.savefig(os.path.join(best_result_path, 'images', f"word_evolution_{topic_name}.png"))
    plt.show()

def visualize_topic_evolution(num_top_words):
    images_path = os.path.join(best_results_path, 'images')
    os.makedirs(images_path, exist_ok=True)
    
    for topic in range(beta.shape[1]):
        topic_name = ''

        # words_index = list(set([np.argmax(beta[time][topic], n=num_top_words) for time in range(len(timelist))]))
        # betas = [[np.nargmax(beta[time][topic][word], n=num_top_words) for time in range(timelist)] for word in words_index]
        words = set([word for time in range(len(timelist)) for word in top_words[time][0].split(' ')])
        betas = [get_betas(word, topic) for word in get_word_index(words)]        
        graphic_(list(words), betas, topic_name, os.path.join(images_path, f'topic_{topic}.png'))

In [ ]:
visualize_topic_evolution(10)

In [ ]:
a = set([word for time in range(len(timelist)) for word in top_words[time][0].split(' ')])

In [ ]:
b = [get_betas(word, 0) for word in get_word_index(a)]
#print(betas)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

best_results_path = './best_results'
def graphic_(words, betas, topic_name, images_path):
    """
    Genera un único gráfico que muestra la evolución de las palabras a lo largo del tiempo.

    Args:
        words: Lista de palabras a graficar.
        betas: Lista de listas, donde cada sublista contiene los valores beta para una palabra en cada timestamp.
        topic_name: Nombre del tema.
        timelist: Lista de timestamps.
        best_result_path: Ruta para guardar el gráfico.
    """
    T = len(timelist)
    
    # Crear una figura grande
    fig, ax1 = plt.subplots(figsize=(20, 10), dpi=80, facecolor='w', edgecolor='k')

    # Graficar las líneas para cada palabra
    for i, comp in enumerate(betas):
        ax1.plot(range(len(timelist)), comp, label=words[i], lw=2, linestyle='--', marker='o', markersize=4)

    # Configurar la leyenda
    ax1.legend(frameon=False, loc='upper center', ncol=5) # Ajusta la ubicación y el número de columnas de la leyenda

    # Configurar las etiquetas del eje x
    ax1.set_xticks(np.arange(len(timelist)))
    ax1.set_xticklabels(timelist, rotation=45, ha='right') # Rota las etiquetas para que sean legibles

    # Configurar el título
    ax1.set_title(f'Evolución de las palabras en el tema: {topic_name}', fontsize=14)

    # Añadir etiquetas a los ejes
    ax1.set_xlabel("Tiempo")
    ax1.set_ylabel("Valor Beta")
    
    # Ajustar el diseño para evitar que se superpongan los elementos
    plt.tight_layout()

    # Guardar el gráfico
    plt.savefig(images_path)

    # Mostrar el gráfico
    plt.show()

In [ ]:
graphic_(list(a), b, 'test')

In [ ]:
!zip -r best_results/images/images.zip /kaggle/working/best_results/images